In [1]:
import sys
sys.path.append('/home/miketheologitis/MapReduce-Implementation')

from mapreduce.cluster.local_cluster import LocalCluster

In [2]:
local_cluster = LocalCluster(
    n_workers=2,
    n_masters=1,
    initialize=True,
    verbose=False
)

In [3]:
local_monitoring = local_cluster.local_monitoring

In [30]:
import random
import string

def generate_random_string(str_len):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(str_len))

def generate_random_string_list(n, str_len):
    return [generate_random_string(str_len) for _ in range(n)]

# Example usage
#data = generate_random_string_list(n=500000, str_len=20)
data = generate_random_string_list(n=1000, str_len=20)

In [31]:
data[:10]

['afgwlmobndycqkbmtpar',
 'ljnjdcmfkyziehkkgcmw',
 'mktzzafbbvuimhixdjrc',
 'wkwxefguznniflcmjdnz',
 'tmwygolsfzrmptwchbiw',
 'abzijzqkaqpjmhflitbs',
 'gbeftolnhaaiwmmmcvhv',
 'tvzlxiijbymrozlpftvy',
 'dxwmjabdmkuuuzcfqhnq',
 'jzqwfbgalktvzqctayad']

In [32]:
import sys
print(f"Size: {round(sys.getsizeof(data) / (1024 * 1024), 3)} MB")

Size: 0.008 MB


In [33]:
def map_func(data):
    result = []
    for string in data:
        for char in string:
            result.append((char, 1))
    return result

def reduce_func(data):
    return sum(data)

In [34]:
future = local_cluster.mapreduce(data, map_func, reduce_func)

In [35]:
future

<Future at 0x7f0624098290 state=running>

In [36]:
#!docker kill 21ebf4c67c9a a41163fe2658 a41163fe2658 a0cf7439958b

In [18]:
local_monitoring.print_zoo()


----------------- Zoo Masters -----------------
Master a5e4233c5c40 :  MasterInfo(state='nothing')

----------------- Zoo Workers -----------------
Worker 7e249a741a7d :  WorkerInfo(state='idle')
Worker aadf463b3d50 :  WorkerInfo(state='idle')
Worker d6bff871c674 :  WorkerInfo(state='idle')
Worker a69b2725051b :  WorkerInfo(state='idle')
Worker f6b4937a34c2 :  WorkerInfo(state='idle')
Worker 6944045d19fe :  WorkerInfo(state='idle')
Worker ca2269b57877 :  WorkerInfo(state='idle')
Worker 0b4bb8d05862 :  WorkerInfo(state='idle')
Worker bf94bffc1544 :  WorkerInfo(state='idle')
Worker 7f746f779506 :  WorkerInfo(state='idle')
Worker 1f6487c2b9a4 :  WorkerInfo(state='idle')
Worker 8374f2cd0cd1 :  WorkerInfo(state='idle')
Worker 30e93d426efa :  WorkerInfo(state='idle')
Worker cd40d855dbb0 :  WorkerInfo(state='idle')
Worker 4bdcbf1af55f :  WorkerInfo(state='idle')
Worker e7d0ef08ce52 :  WorkerInfo(state='idle')

----------------- Zoo Map Tasks -----------------

----------------- Zoo Shuffle T

In [38]:
local_monitoring.print_hdfs('jobs')

job_0/
  data.pickle
  map_func.pickle
  map_results/
    0.pickle
    1.pickle
  map_tasks/
    0.pickle
    1.pickle
  reduce_func.pickle
  reduce_results/
    0_1_2.pickle
    12_13_14.pickle
    15_16_17.pickle
    18_19.pickle
    20_21.pickle
    22_23.pickle
    24_25.pickle
    3_4_5.pickle
    6_7_8.pickle
    9_10_11.pickle
  shuffle_results/
    0.pickle
    1.pickle
    10.pickle
    11.pickle
    12.pickle
    13.pickle
    14.pickle
    15.pickle
    16.pickle
    17.pickle
    18.pickle
    19.pickle
    2.pickle
    20.pickle
    21.pickle
    22.pickle
    23.pickle
    24.pickle
    25.pickle
    3.pickle
    4.pickle
    5.pickle
    6.pickle
    7.pickle
    8.pickle
    9.pickle
job_1/
  data.pickle
  map_func.pickle
  map_results/
    0.pickle
    1.pickle
  map_tasks/
    0.pickle
    1.pickle
  reduce_func.pickle
  reduce_results/
    0_1_2_3_4_5_6_7_8_9_10_11_12.pickle
    13_14_15_16_17_18_19_20_21_22_23_24_25.pickle
  shuffle_results/
    0.pickle
    1.pickl

In [16]:
local_cluster.scale(n_workers=0)

In [17]:
local_cluster.scale(n_workers=10)

In [19]:
local_cluster.shutdown_cluster(cleanup=True)